## ECG Effects of Ranolazine, Dofetilide, Verapamil, and Quinidine

Data: https://physionet.org/content/ecgrdvq/1.0.0/

WFDB makes it easy to read: https://wfdb.readthedocs.io/en/latest/

In [1]:
import wfdb
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd

torch.random.manual_seed(0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Explore the CSV